# Load the Train and test scores and targets

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_scores=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_train_scores.csv")
test_scores=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_scores.csv")

In [3]:
y_train=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\y_train.csv")
y_test=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\y_test.csv")

# Create the datasets with scores and target only

In [12]:
train_scoring=train_scores[["SCORE"]]

In [13]:
train_scoring["TARGET"]=y_train["TARGET"]

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\1283033387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_scoring["TARGET"]=y_train["TARGET"]


In [14]:
train_scoring.head()

,SCORE,TARGET
0,560.941367,0
1,571.797383,0
2,611.326604,0
3,496.948835,0
4,567.188715,0


In [15]:
test_scoring=test_scores[['SCORE']]

In [16]:
test_scoring['TARGET']=y_test['TARGET']

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\1024345474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_scoring['TARGET']=y_test['TARGET']


In [18]:
test_scoring.head()

,SCORE,TARGET
0,567.453859,0
1,576.445644,0
2,555.253204,0
3,602.120594,0
4,535.663997,0


# CREATE DECILES BASED ON SCORES FOR TRAIN

In [32]:
train_scoring["DECILES"], bin_edges=pd.qcut(train_scoring["SCORE"],q=10,retbins=True)

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\626598617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_scoring["DECILES"], bin_edges=pd.qcut(train_scoring["SCORE"],q=10,retbins=True)


In [34]:
train_decile_summary=train_scoring.groupby("DECILES",as_index=False).agg(TOTAL=("SCORE","count"),BADS=("TARGET","sum"))

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\2841987540.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_decile_summary=train_scoring.groupby("DECILES",as_index=False).agg(TOTAL=("SCORE","count"),BADS=("TARGET","sum"))


In [35]:
train_decile_summary

,DECILES,TOTAL,BADS
0,"(474.086, 532.324]",24601,6100
1,"(532.324, 544.177]",24601,3607
2,"(544.177, 552.846]",24601,2592
3,"(552.846, 560.25]",24600,2027
4,"(560.25, 567.256]",24601,1557
5,"(567.256, 574.208]",24601,1265
6,"(574.208, 581.329]",24600,964
7,"(581.329, 589.746]",24668,801
8,"(589.746, 600.82]",24558,556
9,"(600.82, 673.904]",24577,391


In [36]:
train_decile_summary["GOODS"]=train_decile_summary["TOTAL"]-train_decile_summary["BADS"]

In [37]:
train_decile_summary["BAD%"]=train_decile_summary["BADS"]/train_decile_summary["TOTAL"]
train_decile_summary["GOOD%"]=train_decile_summary["GOODS"]/train_decile_summary["TOTAL"]

In [38]:
train_decile_summary["CUM_BAD%"]=train_decile_summary["BADS"].cumsum()/train_decile_summary["BADS"].sum()
train_decile_summary["CUM_GOOD%"]=train_decile_summary["GOODS"].cumsum()/train_decile_summary["GOODS"].sum()

In [42]:
train_decile_summary

,DECILES,TOTAL,BADS,GOODS,BAD%,GOOD%,CUM_BAD%,CUM_GOOD%
0,"(474.086, 532.324]",24601,6100,18501,0.247957,0.752043,0.307150,0.081809
1,"(532.324, 544.177]",24601,3607,20994,0.146620,0.853380,0.488771,0.174642
2,"(544.177, 552.846]",24601,2592,22009,0.105362,0.894638,0.619285,0.271963
3,"(552.846, 560.25]",24600,2027,22573,0.082398,0.917602,0.721349,0.371779
4,"(560.25, 567.256]",24601,1557,23044,0.063290,0.936710,0.799748,0.473677
5,"(567.256, 574.208]",24601,1265,23336,0.051421,0.948579,0.863444,0.576866
6,"(574.208, 581.329]",24600,964,23636,0.039187,0.960813,0.911984,0.681381
7,"(581.329, 589.746]",24668,801,23867,0.032471,0.967529,0.952316,0.786918
8,"(589.746, 600.82]",24558,556,24002,0.022640,0.977360,0.980312,0.893052
9,"(600.82, 673.904]",24577,391,24186,0.015909,0.984091,1.000000,1.000000


# CREATE A FUNCTION TO APPLY THE DECILES FUNCTION ON TEST

In [45]:
def get_decile_summary(dataframe,bin_edges):
    decile_summary=dataframe.copy(deep=True)
    decile_summary["DECILES"]=pd.cut(decile_summary["SCORE"],bins=bin_edges)
    decile_summary=decile_summary.groupby("DECILES", as_index=False).agg(TOTAL=("SCORE","count"),BADS=("TARGET","sum"))
    decile_summary["GOODS"]=decile_summary["TOTAL"]-decile_summary["BADS"]
    decile_summary["BADS%"]=decile_summary["BADS"]/decile_summary["TOTAL"]
    decile_summary["GOODS%"]=decile_summary["GOODS"]/decile_summary["TOTAL"]
    decile_summary["CUM_BAD%"]=decile_summary["BADS"].cumsum()/decile_summary["BADS"].sum()
    decile_summary["CUM_GOOD%"]=decile_summary["GOODS"].cumsum()/decile_summary["GOODS"].sum()
    return decile_summary

In [46]:
test_decile_summary=get_decile_summary(test_scoring,bin_edges)
test_decile_summary

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\1417532177.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  decile_summary=decile_summary.groupby("DECILES", as_index=False).agg(TOTAL=("SCORE","count"),BADS=("TARGET","sum"))


,DECILES,TOTAL,BADS,GOODS,BADS%,GOODS%,CUM_BAD%,CUM_GOOD%
0,"(474.087, 532.324]",6198,1497,4701,0.241530,0.758470,0.301511,0.083151
1,"(532.324, 544.177]",6175,894,5281,0.144777,0.855223,0.481571,0.176560
2,"(544.177, 552.846]",6250,657,5593,0.105120,0.894880,0.613897,0.275488
3,"(552.846, 560.25]",6229,509,5720,0.081715,0.918285,0.716415,0.376663
4,"(560.25, 567.256]",6210,392,5818,0.063124,0.936876,0.795368,0.479571
5,"(567.256, 574.208]",6089,320,5769,0.052554,0.947446,0.859819,0.581612
6,"(574.208, 581.329]",6137,268,5869,0.043670,0.956330,0.913797,0.685422
7,"(581.329, 589.746]",6193,195,5998,0.031487,0.968513,0.953072,0.791513
8,"(589.746, 600.82]",5980,135,5845,0.022575,0.977425,0.980262,0.894899
9,"(600.82, 673.904]",6040,98,5942,0.016225,0.983775,1.000000,1.000000


In [47]:
train_scoring.head()

,SCORE,TARGET,DECILES
0,560.941367,0,"(560.25, 567.256]"
1,571.797383,0,"(567.256, 574.208]"
2,611.326604,0,"(600.82, 673.904]"
3,496.948835,0,"(474.086, 532.324]"
4,567.188715,0,"(560.25, 567.256]"


In [50]:
train_scoring["DECILES"], binned_edges=pd.qcut(train_scoring["SCORE"],q=10,labels=[0,1,2,3,4,5,6,7,8,9],retbins=True)
train_scoring.head()

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\4102232866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_scoring["DECILES"], binned_edges=pd.qcut(train_scoring["SCORE"],q=10,labels=[0,1,2,3,4,5,6,7,8,9],retbins=True)


,SCORE,TARGET,DECILES
0,560.941367,0,4
1,571.797383,0,5
2,611.326604,0,9
3,496.948835,0,0
4,567.188715,0,4


In [51]:
grade_map = {
    0: "E", 1: "E",
    2: "D", 3: "D",
    4: "C", 5: "C",
    6: "B", 7: "B",
    8: "A", 9: "A"
}

train_scoring["GRADE"] = train_scoring["DECILES"].astype(int).map(grade_map)

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\3420374832.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_scoring["GRADE"] = train_scoring["DECILES"].astype(int).map(grade_map)


In [53]:
train_scoring.head()

,SCORE,TARGET,DECILES,GRADE
0,560.941367,0,4,C
1,571.797383,0,5,C
2,611.326604,0,9,A
3,496.948835,0,0,E
4,567.188715,0,4,C


In [54]:
grade_summary_train=train_scoring.groupby("GRADE",as_index=False).agg(TOTAL=("TARGET","count"),BAD=("TARGET","sum"))

In [55]:
grade_summary_train["GOOD"]=grade_summary_train["TOTAL"]-grade_summary_train["BAD"]

In [56]:
grade_summary_train["BAD%"]=grade_summary_train["BAD"]/grade_summary_train["TOTAL"]
grade_summary_train["POPULATION%"]=grade_summary_train["TOTAL"]/grade_summary_train["TOTAL"].sum()

In [57]:
grade_summary_train

,GRADE,TOTAL,BAD,GOOD,BAD%,POPULATION%
0,A,49135,947,48188,0.019273,0.199729
1,B,49268,1765,47503,0.035824,0.200270
2,C,49202,2822,46380,0.057355,0.200002
3,D,49201,4619,44582,0.093880,0.199998
4,E,49202,9707,39495,0.197289,0.200002


In [58]:
grade_summary_train["CUM_BAD%"]=grade_summary_train["BAD"].cumsum()/grade_summary_train["BAD"].sum()
grade_summary_train["CUM_POP%%"]=grade_summary_train["TOTAL"].cumsum()/grade_summary_train["TOTAL"].sum()

In [59]:
grade_summary_train

,GRADE,TOTAL,BAD,GOOD,BAD%,POPULATION%,CUM_BAD%,CUM_POP%%
0,A,49135,947,48188,0.019273,0.199729,0.047684,0.199729
1,B,49268,1765,47503,0.035824,0.200270,0.136556,0.399999
2,C,49202,2822,46380,0.057355,0.200002,0.278651,0.600001
3,D,49201,4619,44582,0.093880,0.199998,0.511229,0.799998
4,E,49202,9707,39495,0.197289,0.200002,1.000000,1.000000


# Apply the same to test

In [61]:
test_scoring.head()

,SCORE,TARGET
0,567.453859,0
1,576.445644,0
2,555.253204,0
3,602.120594,0
4,535.663997,0


In [62]:
test_scoring["DECILES"]=pd.cut(test_scoring["SCORE"],bins=binned_edges,labels=[0,1,2,3,4,5,6,7,8,9])
test_scoring.head()

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\103609315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_scoring["DECILES"]=pd.cut(test_scoring["SCORE"],bins=binned_edges,labels=[0,1,2,3,4,5,6,7,8,9])


,SCORE,TARGET,DECILES
0,567.453859,0,5
1,576.445644,0,6
2,555.253204,0,3
3,602.120594,0,9
4,535.663997,0,1


In [78]:
test_scoring["DECILES"].value_counts(dropna=False)

DECILES
2      6250
3      6229
4      6210
0      6198
7      6193
1      6175
6      6137
5      6089
9      6040
8      5980
NaN       2
Name: count, dtype: int64

*2 test scores fall outside the train bin_edges range.*

In [82]:
#Replace first and last edges:
binned_edges[0] = -np.inf
binned_edges[-1] = np.inf

In [ ]:
#Reapply pd.cut

In [83]:
test_scoring["DECILES"]=pd.cut(test_scoring["SCORE"],bins=binned_edges,labels=[0,1,2,3,4,5,6,7,8,9])
test_scoring.head()

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\103609315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_scoring["DECILES"]=pd.cut(test_scoring["SCORE"],bins=binned_edges,labels=[0,1,2,3,4,5,6,7,8,9])


,SCORE,TARGET,DECILES,GRADE
0,567.453859,0,5,C
1,576.445644,0,6,C
2,555.253204,0,3,A
3,602.120594,0,9,E
4,535.663997,0,1,C


In [84]:
test_scoring["GRADE"]=test_scoring["DECILES"].astype(int).map(grade_map)

C:\Users\owner's\AppData\Local\Temp\ipykernel_19944\2877213039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_scoring["GRADE"]=test_scoring["DECILES"].astype(int).map(grade_map)


In [85]:
test_scoring.head()

,SCORE,TARGET,DECILES,GRADE
0,567.453859,0,5,C
1,576.445644,0,6,B
2,555.253204,0,3,D
3,602.120594,0,9,A
4,535.663997,0,1,E


In [86]:
# Create grade-level summary for TEST

grade_summary_test = (
    test_scoring
        .groupby("GRADE", as_index=False)
        .agg(
            TOTAL=("SCORE", "count"),
            BAD=("TARGET", "sum")
        )
)

# Calculate GOODS
grade_summary_test["GOOD"] = grade_summary_test["TOTAL"] - grade_summary_test["BAD"]

# Bad rate
grade_summary_test["BAD%"] = grade_summary_test["BAD"] / grade_summary_test["TOTAL"]

# Population %
grade_summary_test["POPULATION%"] = grade_summary_test["TOTAL"] / grade_summary_test["TOTAL"].sum()

# Sort grades properly (A → E)
grade_summary_test = grade_summary_test.sort_values("GRADE").reset_index(drop=True)

grade_summary_test

,GRADE,TOTAL,BAD,GOOD,BAD%,POPULATION%
0,A,12022,233,11789,0.019381,0.195470
1,B,12330,463,11867,0.037551,0.200478
2,C,12299,712,11587,0.057891,0.199974
3,D,12479,1166,11313,0.093437,0.202901
4,E,12373,2391,9982,0.193243,0.201177
